In [1]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import os.path
from matplotlib import pyplot
%matplotlib inline
pyplot.rcParams['figure.figsize'] = [24, 16]
vid = '/home/stridera/code/robotron/resources/video/robotron.mp4';


In [2]:
def numberCleanup(image):
    binary_output = np.zeros_like(image)
    binary_output[image >= 30] = 1
    return binary_output

In [3]:
import hashlib

def h11(w):
    return hashlib.md5(w).hexdigest()[:9].lower()

def h6(w):
    h = hashlib.md5(w)
    return h.digest().encode('base64')[:6].lower()

hashmap = {}
def getDigitFromImageBinary(score_bin):
    digest = h6(score_bin)
    if digest in NUMBERS_HASH:
        return NUMBERS_HASH[digest]
    
    digit = -1
    flat = score_bin.flatten()
    results = np.apply_along_axis(lambda row: np.sum(row == flat), 1, NUMBERS)
    before = np.copy(results)
    results[results < 140] = 0 # Max 168
    
    if np.count_nonzero(results) > 0:
        digit = np.argmax(results)    
        while digit > 9:
            digit -= 10

    if digest not in hashmap:
        print(score_bin)
        print(digit, 'digest', digest, 'argmax', np.argmax(results))       
        print("")
        hashmap[digest] = 1
        
    return digit


In [4]:
def getScore(gray):
    score = 0
    tl = np.copy(SCOREBOX)
    br = tl + SCORE_SIZE
    score_img = gray[tl[1]:br[1], tl[0]:br[0]]
    score_bin = numberCleanup(score_img)

    place = 0
    while (np.count_nonzero(score_bin)):
        digit = getDigitFromImageBinary(score_bin)
        if digit == -1:
            return -1

        score += digit * (10 ** place)

        tl[0] = tl[0] - SCORE_SIZE[0] - SCORE_BUFFER
        br[0] = tl[0] + SCORE_SIZE[0]
        score_img = gray[tl[1]:br[1], tl[0]:br[0]]
        score_bin = numberCleanup(score_img)
        place += 1;

    return score

In [5]:
def overlayText(image, text, location, size=3, weight=8, color=(255,255,255)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image, text, location, font, size, color, weight)
    return image

In [6]:
def addSquares(image, locx, locy, color=(0,255,255)):
    cv2.rectangle(image, (locx[0], locx[1]), (locy[0], locy[1]), color, 2)

In [7]:
def writeScoreToImage(image):
    score = getScore(image)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return overlayText(image, "Score: {}".format(score), (25,50), 1, 4)

In [8]:
NUMBERS = [
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1],
    [0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0],
    [0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0],
    [0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0],
    [1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1],
]

In [17]:
numbers_hash = {
	'wyj8ay':0,'warr4l':0,'bbbnw5':0,'trkkxr':0,'1jiwqc':0,'ceh3iu':0,'r4swfw':0,'3dbr5o':0,'dqos4u':0,'kx9cgf':0,'g5c/iv':0,
	'tldks7':1,'szcrhd':1,'wf5wgw':1,'9etfwr':1,'ykoeur':1,'zfctly':1,'xhsqw2':1,'tnoezf':1,'cewbyp':1,'n/z8cd':1,'d+qvoc':1,'+ugtdk':1,
	'rht93c':2,'042urk':2,'grbo36':2,'c90lxp':2,'5j34un':2,'ohcpnt':2,'ujhlc7':2,'qulngk':2,'vrvp7j':2,'eerjiq':2,'fydmir':2,'wi8rxd':2,'cdyngv':2,'f751x+':2,'/w4kld':2,
	'vwujcu':3,'oqca4r':3,'jefo8a':3,'7jjfwq':3,'vbpjip':3,'+etzhn':3,'c/+zoa':3,'j4a5z5':3,'e4badr':3,'u2f8cd':3,
	'yskjgx':4,'vmjz8y':4,'pvased':4,'lpssgx':4,'ijtxus':4,'vmjz8y':4,'pvased':4,'lpssgx':4,'bgcvif':4,'2zmxyl':4,'kndv5o':4,'8dbv8b':4,'hsgm9s':4,
	'ssop8b':5,'nbr42g':5,'b2ngl9':5,'t6x5lz':5,'1jveha':5,'zt2j2m':5,'xrloea':5,'bosspd':5,'u7sc9o':5,'ocvril':5,'s//eqz':5,'4vz+wc':5,'+jpqpv':5,'hvctsc':5,'sl4ogs':5,'i6zcip':5,'44a0cs':5,'fqrmvx':5,'dxbmqj':5,
	'bynaqu':6,'aj2mo7':6,'obnnen':6,'xftabr':6,'fldafz':6,
	'jst1wf':7,'vhfsgy':7,'49f4x9':7,'fg06b2':7,'5ms0tb':7,'tsfu0q':7,'ttmg5e':7,'8weloy':7,'3hen02':7,'ryupe5':7,'ot26gj':7,'rignx4':7,'x/pesb':7,
	'sqyncb':8,'bvirlr':8,'g559ta':8,'hpn6iu':8,'ednnrk':8,
	'tosdv5':9,'ronwec':9,'sldlii':9,'clibho':9,'dj6rbu':9,'jhnpc2':9,

    # catch non-numbers... perhaps we can use these to know when we're transitioning
	'zhxyap':-1,'90dd+e':-1,'/ao8j8':-1,'ywgemb':-1,'mpejnl':-1,'gccexm':-1,'jly+ic':-1,'8dribl':-1,'fxazsm':-1,'+ieq3f':-1,'scd6pq':-1,'kme+xy':-1,'vdp0zy':-1,'at2zis':-1,'fl0/fk':-1,'rzcihe':-1,'nyfcla':-1,'r/r6fa':-1,'swemgn':-1,'elv4lf':-1,'sx8zxm':-1,'jjahka':-1,'ynpx+1':-1,'+3zmlu':-1,'/jaxop':-1,'wkyk9e':-1,'jsyya6':-1,'vekwkn':-1,'gg62jt':-1,'/71uxu':-1,'ydtn2e':-1,'osmjkf':-1,'0kyeh6':-1,'jndem2':-1,'1+zyj2':-1,'s3down':-1,'ms5h7e':-1,'404njd':-1,'r1vdh+':-1,'vi+jsn':-1,'/j4kco':-1,'hq/v6g':-1,'llpnnx':-1,'xbkfod':-1
}

In [18]:
SCOREBOX = np.array([479, 91])
SCORE_SIZE = np.array([12, 14])
SCORE_BUFFER = 2

cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
while(True):
    # Capture frame-by-frame
    ret, image = cap.read()
    # Our operations on the frame come here
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    score = getScore(gray)
    overlayText(image, "Score: {}".format(score), (25,50), 1, 4)
    
    # Display the resulting frame 
    cv2.imshow('frame', image)
    if cv2.waitKey(1) & 0xFF == ord('q'): # wait for 1 millisecond
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
# print(hashmap)